# Causal Evaluation 
> Using causal analysis to compute the influence of T on Y

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

<ipython-input-1-ac09909db896>:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
#! pip install dowhy
from dowhy import CausalModel
import dowhy.datasets

In [3]:
from pathlib import Path
import csv
import seaborn as sns; sns.set_theme()
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import functools

pd.options.display.float_format = '{:.2f}'.format

In [4]:
import seaborn as sns
sns.set_theme(style="darkgrid")


In [5]:
%matplotlib inline

## Load Data Samples

In [6]:
#Load data samples
eval_df = pd.read_csv(
    '/workspaces/CodeCheckList/data/linguistic_capabilities/huggingface-CodeBERTa-small-v1_100.0_20.csv',
    index_col=0
    )

In [7]:
eval_df.head(2)

,sample_id,ast_element,sample,masking_rate,numper_of_masked_tokens,ast_element_ocurrences,mask_jaccard,mask_sorensen_dice,mask_levenshtein,mask_random_avg_jaccard,...,mask_random_std_jaccard,mask_random_std_sorensen_dice,mask_random_std_levenshtein,n_ast_errors,ast_levels,n_whitespaces_,complexity,nloc,token_counts,n_ast_nodes
0,0,comparison_operator,def test_should_generate_secret_with_specified...,1.00,20,1,0.85,0.92,0.90,0.98,...,0.03,0.01,0.02,0,15,29,1,15,50,96
1,0,identifier,def test_should_generate_secret_with_specified...,1.00,24,9,0.82,0.90,0.83,0.94,...,0.05,0.03,0.04,0,15,29,1,15,50,96


In [9]:
eval_df.columns

Index(['sample_id', 'ast_element', 'sample', 'masking_rate',
       'numper_of_masked_tokens', 'ast_element_ocurrences', 'mask_jaccard',
       'mask_sorensen_dice', 'mask_levenshtein', 'mask_random_avg_jaccard',
       'mask_random_avg_sorensen_dice', 'mask_random_avg_levenshtein',
       'mask_random_std_jaccard', 'mask_random_std_sorensen_dice',
       'mask_random_std_levenshtein', 'n_ast_errors', 'ast_levels',
       'n_whitespaces_', 'complexity', 'nloc', 'token_counts', 'n_ast_nodes'],
      dtype='object')

In [36]:
eval_df.shape[0]

In [67]:
data_dict = {
    'ast_concept' : np.concatenate((eval_df.ast_element.values, eval_df.ast_element.values), axis=None),
    't_binary_treatment' : [True] * eval_df.shape[0] + [False] * eval_df.shape[0],
    'y_jaccard' :  np.concatenate((eval_df.mask_jaccard.values, eval_df.mask_random_avg_jaccard.values), axis=None),
    'y_levenshtein' :  np.concatenate((eval_df.mask_levenshtein.values, eval_df.mask_random_avg_levenshtein.values), axis=None),
    'y_sorensen' :  np.concatenate((eval_df.mask_sorensen_dice.values, eval_df.mask_random_std_sorensen_dice.values), axis=None),
    'z_n_ast_errors' : np.concatenate((eval_df.n_ast_errors.values, eval_df.n_ast_errors.values), axis=None),
    'z_ast_levels' : np.concatenate((eval_df.ast_levels.values, eval_df.ast_levels.values), axis=None),
    'z_n_whitespaces' : np.concatenate((eval_df.n_whitespaces_.values, eval_df.n_whitespaces_.values), axis=None),
    'z_complexity' : np.concatenate((eval_df.complexity.values, eval_df.complexity.values), axis=None),
    'z_nloc' : np.concatenate((eval_df.nloc.values, eval_df.nloc.values), axis=None),
    'z_token_counts' : np.concatenate((eval_df.token_counts.values, eval_df.token_counts.values), axis=None),
    'z_n_ast_nodes' : np.concatenate((eval_df.n_ast_nodes.values, eval_df.n_ast_nodes.values), axis=None)
}

In [68]:
causal_data_df = pd.DataFrame(data=data_dict)

In [69]:
causal_data_df.head()

,ast_concept,t_binary_treatment,y_jaccard,y_levenshtein,y_sorensen,z_n_ast_errors,z_ast_levels,z_n_whitespaces,z_complexity,z_nloc,z_token_counts,z_n_ast_nodes
0,comparison_operator,True,0.85,0.90,0.92,0,15,29,1,15,50,96
1,identifier,True,0.82,0.83,0.90,0,15,29,1,15,50,96
2,string,True,0.42,0.39,0.60,0,15,29,1,15,50,96
3,parameters,True,0.86,0.88,0.93,0,15,29,1,15,50,96
4,comparison_operator,True,0.50,0.51,0.67,0,15,29,1,17,46,90


In [70]:
causal_data_df.describe()

,y_jaccard,y_levenshtein,y_sorensen,z_n_ast_errors,z_ast_levels,z_n_whitespaces,z_complexity,z_nloc,z_token_counts,z_n_ast_nodes
count,91226.00,91226.00,91226.00,91226.00,91226.00,91226.00,91226.00,91226.00,91226.00,91226.00
mean,0.83,0.83,0.46,0.05,11.58,39.66,2.58,12.71,87.94,143.63
std,0.19,0.19,0.42,0.30,3.04,33.41,2.56,10.55,70.49,108.94
min,0.00,0.00,0.00,0.00,4.00,2.00,1.00,1.00,5.00,8.00
25%,0.74,0.74,0.03,0.00,10.00,15.00,1.00,5.00,36.00,59.00
50%,0.89,0.90,0.25,0.00,11.00,29.00,1.00,10.00,69.00,114.00
75%,0.98,0.99,0.91,0.00,13.00,55.00,3.00,18.00,121.00,202.00
max,1.00,1.00,1.00,19.00,27.00,214.00,45.00,171.00,1130.00,648.00


## I. Create a causal model from the data and given graph.


In [71]:
causal_model = CausalModel(
    data = causal_data_df,
    treatment = ['t_binary_treatment'],
    outcome = ['y_levenshtein'],
    common_causes = ['z_token_counts','z_nloc']
)

## II. Identify causal effect and return target estimands.


In [72]:
identified_estimand = causal_model.identify_effect()

# III. Estimate the target estimand using a statistical method.


In [73]:
estimate = causal_model.estimate_effect(identified_estimand,
                                 method_name="backdoor.propensity_score_matching")

In [ ]:
estimate

# IV. Refute the obtained estimate using multiple robustness checks.

In [74]:
refute_results = causal_model.refute_estimate(
    identified_estimand, 
    estimate,
    method_name="random_common_cause"
    )

In [ ]:
refute_results